## Scrape cars on sale on various trader websites

The code below was meant to scrape all cars available on autotrader.com, and store them in a sql databases for further analyses. This particular trader seems to be actively fighting scraping of its offers by regularly modifying (hour basis it seems...) the html format and css files and class names. Which makes the current script useless. Will come back to this in a bit to figure a strategy.

(Is it possible that the class names are randomly generated?)

The next steps once this issue is resolved:
* Load in list of zipcodes (data/zipcode.sqlite)
* Run through all pages of the search (browser.find_element_by_class_name('pagination-button-next').click())
* Run through all zipcodes.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sqlite3

In [ ]:
# Connect to dn
conn = sqlite3.connect('data/sales.db')

In [ ]:
# %qtconsole

trader = 'http://www.autotrader.com'
zipcode = 10005
browser = webdriver.Chrome()
browser.get(trader)
delay = 3 # seconds


elem = browser.find_element_by_name('fyc-form-j_id_bb-j_id_bn-j_id_cp-zipcode')  # Find the search box
elem.send_keys(Keys.ESCAPE)
elem.send_keys(str(zipcode))
elem.send_keys(Keys.RETURN)


In [ ]:
soup = BeautifulSoup(browser.page_source, "lxml")

# Create DataFrame
# columns = ['NewUsed', 'Year', 'Make', 'Type','Mileage','Price','ZipCode','Source']
all_cars = pd.DataFrame() #columns=columns)

#Make and Type
brands = soup.find_all("span", class_="atcui-truncate ymm")
#Price and Mileage
details = soup.find_all("div", class_="atcui-section atcui-clearfix   listing-content ")

for cars, entry in zip(brands, details):
    carsSTR = cars.text.split()
    car_nu = carsSTR[0]
    car_year = int(carsSTR[1])
    car_make = carsSTR[2]
    car_type = " ".join(carsSTR[3:])
    
    try:
        milesSTR = entry.find_all("span", class_="mileage")[0].text
        car_miles = int(milesSTR.split()[0].replace(',',''))
    except:
        car_miles = 'Null'
    
    try:
        priceSTR = entry.find_all("h4", class_="primary-price")[0].text
        car_price = int(priceSTR.replace('$','').replace(',',''))
    except:
        car_price = 'Null'
    
    
    print('{} {} {} {} {} {}'.format(car_year, car_nu, car_make, car_type, car_price, car_miles))
    all_cars = all_cars.append(pd.DataFrame([[car_year, car_nu, car_make, 
                                              car_type, car_price, car_miles,
                                              zipcode, trader]]),
                              ignore_index = True)
    
all_cars.columns = ['new_used', 'year', 'make', 'type', 'mileage', 'price', 'zipcode', 'source']    
# Update index
index_max = conn.execute('SELECT MAX(car_id) FROM sales').fetchall()
if index_max[0][0] is not None:
    all_cars.index = all_cars.index + 1 + index_max[0][0]
all_cars.to_sql('sales',conn,flavor='sqlite',if_exists='append',index_label='car_id')




In [ ]:
browser.find_element_by_class_name('pagination-button-next').click()


In [ ]:
browser.quit()

In [ ]:
conn.close()